<a href="https://colab.research.google.com/github/bees4ever/ai-workshop-2022/blob/main/YOLO_Workshop_Capgemini_2022_Exercise_02_ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crime Detection Workshop
*Capgemini Challenge*

Catch the thief through image analysis and prove your analytical skill!

Estimated Duration: 20 min

Challange Points: 2 

## Story - Spies in the shopping mall 

According to the intelligence service, there was a spy in the shopping mall during regular opening hours. This spy was conspicuously dressed. You are to automatically analyze the images of the surveillance camera for conspicuously dressed persons. The image of the suspicious person is to be handed over to the secret service.
To do this, create the crops of all the people and then extract the three most common colors in the image. If a conspicuous color occurs frequently, it is a suspicious person.


Note: The data are downloaded from kaggle and underly different lizenses, even unkown.

- https://www.kaggle.com/datasets/odins0n/ucf-crime-dataset (CC0 lizense)
- https://www.kaggle.com/datasets/fmena14/crowd-counting (unkown)



Step 1: Dataset
Load the dataset with following command:

In [ ]:
!wget https://github.com/bees4ever/ai-workshop-2022/raw/main/surveillance_dataset/workshop_task_02_dataset.zip

In [ ]:
!unzip workshop_task_02_dataset.zip

## Step 2: Load YOLO

In [ ]:
import torch
# Model
model = torch.hub.load('ultralytics/yolov3', 'yolov3')

In [ ]:
# Load YOLO v5
!git clone  'https://github.com/ultralytics/yolov5.git'

In [ ]:
!pip install -qr '/content/yolov5/requirements.txt'  # install dependencies

## Step 3: Dominant Color



In [ ]:
import scipy.cluster
import sklearn.cluster
import numpy
from PIL import Image

def dominant_colors(image):  # PIL image input

    image = image.resize((150, 150))      # optional, to reduce time
    ar = numpy.asarray(image)
    shape = ar.shape
    ar = ar.reshape(numpy.product(shape[:2]), shape[2]).astype(float)

    kmeans = sklearn.cluster.MiniBatchKMeans(
        n_clusters=3,
        init="k-means++",
        max_iter=20,
        random_state=1000
    ).fit(ar)
    codes = kmeans.cluster_centers_

    vecs, _dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
    counts, _bins = numpy.histogram(vecs, len(codes))    # count occurrences

    colors = []
    for index in numpy.argsort(counts)[::-1]:
        colors.append(tuple([int(code) for code in codes[index]]))
    return colors   


In [ ]:
#' Run object detection
!python yolov5/detect.py --source /content/workshop_task_02_dataset  --save-crop

In [ ]:
#' prepare dataset and extract person object
#' TODO for students - write a loop

import progressbar as pb
import os
for img in pb.progressbar(os.listdir('workshop_task_02_dataset')):
    img_path = os.path.join('workshop_task_02_dataset', img) 
    #' TODO this part
    results = model(img_path)
    cropped = results.crop(save=True)






In [ ]:
#'TODO: Define extracted folder path, the provided one might be wrong!
ext_data_path = 'yolov5/runs/detect/exp2/crops/person'


In [ ]:
#' TODO please finish here the function to assign the corresponsing data sextractions to the Data frame

def append_data_to_df(df_person_dominant_color, img_path, seq_file):
  # pass
    

In [ ]:
#' create data frame with persons per sequence and dominator color
import re
import os
import pandas as pd
import progressbar as pb
df_person_dominant_color = {"image": [], "seq_number": [], "dominant_color_1": [], "dominant_color_2": [], "dominant_color_3": []}

for img_name in pb.progressbar(os.listdir(ext_data_path)):
  main_dir = ext_data_path
  seq_image = os.path.join(main_dir, img_name)
  #' TODO: finish the function prepared above
  append_data_to_df(df_person_dominant_color, seq_image, seq_image)
    
   

df_person_dominant_color = pd.DataFrame(df_person_dominant_color)

In [ ]:
df_person_dominant_color = pd.DataFrame(df_person_dominant_color)

In [ ]:
#' preview dataset
df_person_dominant_color.head(30)

In [ ]:
import base64
def imgToHtml(img):
  with open(img, 'rb') as binary_file:
     binary_file_data = binary_file.read()
     base64_encoded_data = base64.b64encode(binary_file_data)
     base64_message = base64_encoded_data.decode('utf-8')
     return '<img width="50px" src="data:image/png;base64,' + base64_message + '"/>'



In [ ]:
#' some example of explorative data analysis
from IPython.core.display import display, HTML
from google.colab import files
#' show colors in nice bars
def colorToHtml(color):
  return '<div style="display: inline-block; width: 40px; height: 40px; background-color: rgb('+ str(color[0]) +','+ str(color[1] )+',' + str(color[2])+ ');"></div>'


html_str = ''
for (index, row) in df_person_dominant_color.head(10).iterrows():
  html_str += '<b>' + str(row.seq_number) + '</b>' + imgToHtml(row.image) + colorToHtml(row.dominant_color_1) + colorToHtml(row.dominant_color_2) + colorToHtml(row.dominant_color_3)


display(HTML(html_str))

## Step 4 - Filter for suspicoius colors



In [ ]:
!pip install webcolors

In [ ]:
from webcolors import rgb_to_name

from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)
#' convert a rgb color code to a name
#' which reduces the three dimension of rgb to
#' one dimension
def convert_rgb_to_names(rgb_tuple):
    
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return names[index]

In [ ]:
#' predefined
suspicious_colors = ['aquamarine',
 'azure',
 'blanchedalmond',
 'blue',
 'blueviolet',
 'chartreuse',
 'coral',
 'cornflowerblue',
 'cornsilk',
 'crimson',
 'cyan',
 'darkblue',
 'darkgoldenrod',
 'darkgreen',
 'darkmagenta',
 'darkorange',
 'darkcyan',
 'darkred',
 'darksalmon',
 'darkseagreen',
 'darkviolet',
 'deeppink',
 'deepskyblue',
 'dodgerblue',
 'firebrick',
 'floralwhite',
 'forestgreen',
 'greenyellow',
 'honeydew',
 'hotpink',
 'indigo',
 'khaki',
 'lavenderblush',
 'lawngreen',
 'lemonchiffon',
 'lightcoral',
 'lightgoldenrodyellow',
 'lightgreen',
 'lightsalmon',
 'lightseagreen',
 'lightskyblue',
 'lightyellow',
 'limegreen',
 'magenta',
 'mediumaquamarine',
 'mediumblue',
 'mediumorchid',
 'mediumpurple',
 'mediumslateblue',
 'mediumspringgreen',
 'mediumturquoise',
 'mediumvioletred',
 'midnightblue',
 'moccasin',
 'navajowhite',
 'oldlace',
 'olivedrab',
 'orange',
 'orangered',
 'orchid',
 'palegoldenrod',
 'palegreen',
 'palevioletred',
 'papayawhip',
 'peachpuff',
 'pink',
 'plum',
 'royalblue',
 'salmon',
 'seagreen',
 'skyblue',
 'slateblue',
 'springgreen',
 'steelblue',
 'tomato',
 'turquoise',
 'violet',
 'yellow',
 'yellowgreen']


In [ ]:
#' TODO: run through all images and find the image which has a color in the list of suspicious colors
images_found = set()
for (index, row) in pb.progressbar(df_person_dominant_color.iterrows(), max_value=len(df_person_dominant_color)):
  pass

In [ ]:
for img in images_found:
  print(img)
  display(HTML(imgToHtml(img)))
